# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [3]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2019-03-15 17:10:02,638 : INFO : reading file reviews_data.txt.gz...this may take a while
2019-03-15 17:10:02,639 : INFO : read 0 reviews
2019-03-15 17:10:04,166 : INFO : read 10000 reviews
2019-03-15 17:10:05,709 : INFO : read 20000 reviews
2019-03-15 17:10:07,474 : INFO : read 30000 reviews
2019-03-15 17:10:09,122 : INFO : read 40000 reviews
2019-03-15 17:10:10,928 : INFO : read 50000 reviews
2019-03-15 17:10:12,682 : INFO : read 60000 reviews
2019-03-15 17:10:14,338 : INFO : read 70000 reviews
2019-03-15 17:10:15,707 : INFO : read 80000 reviews
2019-03-15 17:10:17,147 : INFO : read 90000 reviews
2019-03-15 17:10:18,525 : INFO : read 100000 reviews
2019-03-15 17:10:19,893 : INFO : read 110000 reviews
2019-03-15 17:10:21,269 : INFO : read 120000 reviews
2019-03-15 17:10:22,693 : INFO : read 130000 reviews
2019-03-15 17:10:24,196 : INFO : read 140000 reviews
2019-03-15 17:10:25,581 : INFO : read 150000 reviews
2019-03-15 17:10:27,385 : INFO : read 160000 reviews
2019-03-15 17:10:28,775

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [4]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-03-15 17:10:48,365 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-03-15 17:10:48,366 : INFO : collecting all words and their counts
2019-03-15 17:10:48,367 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-15 17:10:48,581 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2019-03-15 17:10:48,788 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2019-03-15 17:10:49,032 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2019-03-15 17:10:49,258 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2019-03-15 17:10:49,511 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2019-03-15 17:10:49,758 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2019-03-15 17:10:49,966 : INFO : PROGRESS: a

2019-03-15 17:11:23,995 : INFO : EPOCH 2 - PROGRESS: at 36.82% examples, 1473200 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:11:25,002 : INFO : EPOCH 2 - PROGRESS: at 42.33% examples, 1477839 words/s, in_qsize 20, out_qsize 1
2019-03-15 17:11:26,008 : INFO : EPOCH 2 - PROGRESS: at 47.56% examples, 1477010 words/s, in_qsize 17, out_qsize 1
2019-03-15 17:11:27,008 : INFO : EPOCH 2 - PROGRESS: at 52.64% examples, 1477483 words/s, in_qsize 13, out_qsize 6
2019-03-15 17:11:28,009 : INFO : EPOCH 2 - PROGRESS: at 57.88% examples, 1481851 words/s, in_qsize 17, out_qsize 2
2019-03-15 17:11:29,010 : INFO : EPOCH 2 - PROGRESS: at 62.99% examples, 1481877 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:11:30,018 : INFO : EPOCH 2 - PROGRESS: at 68.30% examples, 1483429 words/s, in_qsize 20, out_qsize 0
2019-03-15 17:11:31,018 : INFO : EPOCH 2 - PROGRESS: at 73.15% examples, 1482078 words/s, in_qsize 18, out_qsize 0
2019-03-15 17:11:32,029 : INFO : EPOCH 2 - PROGRESS: at 77.83% examples, 1479432

2019-03-15 17:12:18,950 : INFO : EPOCH 4 - PROGRESS: at 98.13% examples, 1481833 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:12:19,277 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-03-15 17:12:19,281 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-03-15 17:12:19,282 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-03-15 17:12:19,282 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-03-15 17:12:19,291 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-03-15 17:12:19,292 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-03-15 17:12:19,304 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-15 17:12:19,305 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-15 17:12:19,307 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-15 17:12:19,312 : INFO : worker thread finished; awaiting 

2019-03-15 17:13:01,733 : INFO : EPOCH - 1 : training on 41519358 raw words (30351281 effective words) took 21.0s, 1445619 effective words/s
2019-03-15 17:13:02,739 : INFO : EPOCH 2 - PROGRESS: at 4.07% examples, 1255211 words/s, in_qsize 18, out_qsize 5
2019-03-15 17:13:03,741 : INFO : EPOCH 2 - PROGRESS: at 8.62% examples, 1331641 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:13:04,741 : INFO : EPOCH 2 - PROGRESS: at 12.36% examples, 1356673 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:13:05,752 : INFO : EPOCH 2 - PROGRESS: at 16.56% examples, 1374259 words/s, in_qsize 20, out_qsize 1
2019-03-15 17:13:06,755 : INFO : EPOCH 2 - PROGRESS: at 19.91% examples, 1350178 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:13:07,759 : INFO : EPOCH 2 - PROGRESS: at 23.10% examples, 1311898 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:13:08,771 : INFO : EPOCH 2 - PROGRESS: at 27.26% examples, 1307713 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:13:09,775 : INFO : EPOCH 2 - PROGRESS: at 

2019-03-15 17:13:56,033 : INFO : EPOCH 4 - PROGRESS: at 45.24% examples, 1415602 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:13:57,041 : INFO : EPOCH 4 - PROGRESS: at 50.23% examples, 1416196 words/s, in_qsize 19, out_qsize 7
2019-03-15 17:13:58,041 : INFO : EPOCH 4 - PROGRESS: at 55.10% examples, 1419965 words/s, in_qsize 20, out_qsize 4
2019-03-15 17:13:59,041 : INFO : EPOCH 4 - PROGRESS: at 60.14% examples, 1422484 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:00,042 : INFO : EPOCH 4 - PROGRESS: at 65.24% examples, 1422459 words/s, in_qsize 20, out_qsize 0
2019-03-15 17:14:01,050 : INFO : EPOCH 4 - PROGRESS: at 69.97% examples, 1423143 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:02,056 : INFO : EPOCH 4 - PROGRESS: at 74.90% examples, 1424101 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:03,059 : INFO : EPOCH 4 - PROGRESS: at 79.48% examples, 1425667 words/s, in_qsize 20, out_qsize 2
2019-03-15 17:14:04,068 : INFO : EPOCH 4 - PROGRESS: at 84.39% examples, 1428896

2019-03-15 17:14:49,743 : INFO : EPOCH 6 - PROGRESS: at 69.26% examples, 1048125 words/s, in_qsize 20, out_qsize 2
2019-03-15 17:14:50,760 : INFO : EPOCH 6 - PROGRESS: at 73.37% examples, 1055439 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:51,762 : INFO : EPOCH 6 - PROGRESS: at 78.09% examples, 1074611 words/s, in_qsize 20, out_qsize 2
2019-03-15 17:14:52,787 : INFO : EPOCH 6 - PROGRESS: at 82.55% examples, 1085431 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:53,795 : INFO : EPOCH 6 - PROGRESS: at 87.04% examples, 1095803 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:54,795 : INFO : EPOCH 6 - PROGRESS: at 90.65% examples, 1092724 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:55,800 : INFO : EPOCH 6 - PROGRESS: at 94.35% examples, 1092117 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:56,805 : INFO : EPOCH 6 - PROGRESS: at 99.23% examples, 1103746 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:14:56,935 : INFO : worker thread finished; awaiting finish of 9 mo

2019-03-15 17:15:43,568 : INFO : EPOCH 8 - PROGRESS: at 84.39% examples, 1218901 words/s, in_qsize 19, out_qsize 2
2019-03-15 17:15:44,577 : INFO : EPOCH 8 - PROGRESS: at 87.23% examples, 1201040 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:15:45,579 : INFO : EPOCH 8 - PROGRESS: at 90.62% examples, 1190326 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:15:46,609 : INFO : EPOCH 8 - PROGRESS: at 95.44% examples, 1197644 words/s, in_qsize 20, out_qsize 3
2019-03-15 17:15:47,460 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-03-15 17:15:47,463 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-03-15 17:15:47,473 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-03-15 17:15:47,474 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-03-15 17:15:47,478 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-03-15 17:15:47,498 : INFO : worker thread finished; awaiting finish of 4 more th

2019-03-15 17:16:38,893 : INFO : EPOCH 10 - PROGRESS: at 79.72% examples, 1050633 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:16:39,899 : INFO : EPOCH 10 - PROGRESS: at 82.75% examples, 1044759 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:16:40,922 : INFO : EPOCH 10 - PROGRESS: at 85.94% examples, 1041698 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:16:41,936 : INFO : EPOCH 10 - PROGRESS: at 91.08% examples, 1056164 words/s, in_qsize 19, out_qsize 0
2019-03-15 17:16:42,941 : INFO : EPOCH 10 - PROGRESS: at 95.85% examples, 1068663 words/s, in_qsize 17, out_qsize 2
2019-03-15 17:16:43,708 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-03-15 17:16:43,710 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-03-15 17:16:43,712 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-03-15 17:16:43,723 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-03-15 17:16:43,728 : INFO : worker thread finished;

(303491888, 415193580)

In [14]:
model.save("word2vec.model")

2019-03-15 17:23:04,440 : INFO : saving Word2Vec object under word2vec.model, separately None
2019-03-15 17:23:04,443 : INFO : storing np array 'vectors' to word2vec.model.wv.vectors.npy
2019-03-15 17:23:04,481 : INFO : not storing attribute vectors_norm
2019-03-15 17:23:04,481 : INFO : storing np array 'syn1neg' to word2vec.model.trainables.syn1neg.npy
2019-03-15 17:23:04,507 : INFO : not storing attribute cum_table
2019-03-15 17:23:04,634 : INFO : saved word2vec.model


In [16]:
model['good']

/home/nagra-p/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-9.84803140e-01,  2.00643569e-01,  5.01873302e+00,  1.17632627e+00,
       -5.69037721e-02, -2.98790812e-01, -3.36786842e+00,  1.83894411e-01,
       -8.36263120e-01, -1.74371469e+00,  2.84413385e+00, -5.46091139e-01,
        1.71670055e+00,  5.68743324e+00, -1.21971440e+00,  4.10626745e+00,
       -2.16679168e+00,  2.62755990e+00, -3.21339607e+00, -3.73317885e+00,
        2.33368683e+00,  1.34934640e+00,  1.18614173e+00, -1.35659873e+00,
        1.60746694e+00, -2.86103654e+00, -4.39756584e+00, -9.01412293e-02,
       -1.48805439e+00, -2.22964835e+00, -2.99705911e+00,  2.48890936e-01,
       -2.63185835e+00,  1.43324566e+00, -2.87168241e+00,  4.55185795e+00,
       -1.13603115e+00,  2.94469404e+00, -6.78085625e-01, -7.97326803e-01,
        1.49115717e+00, -4.09721470e+00,  3.45179248e+00,  2.17270970e+00,
       -2.68589115e+00,  4.11349630e+00,  2.97663569e-01, -1.53284514e+00,
       -2.81973720e+00, -6.22105122e-01,  4.22532511e+00,  6.00296688e+00,
       -5.69216108e+00,  

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [6]:

w1 = "fall"
model.wv.most_similar(positive=w1)


[('fell', 0.6529394388198853),
 ('falling', 0.5994173288345337),
 ('fallen', 0.5769830942153931),
 ('kick', 0.5465656518936157),
 ('bump', 0.527692973613739),
 ('wind', 0.5063945055007935),
 ('jump', 0.5036382675170898),
 ('lie', 0.49928027391433716),
 ('survive', 0.4979685842990875),
 ('cut', 0.4975726902484894)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [7]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9269418716430664),
 ('friendly', 0.824870765209198),
 ('professional', 0.8123276233673096),
 ('cordial', 0.7933465242385864),
 ('attentive', 0.7925757765769958),
 ('personable', 0.7628620862960815)]

In [11]:
# look up top 6 words similar to 'france'
w1 = ["java"]
model.wv.most_similar (positive=w1,topn=6)


[('latte', 0.6595109701156616),
 ('pastry', 0.6219616532325745),
 ('muffin', 0.6001742482185364),
 ('cappucino', 0.5951939821243286),
 ('cappuccino', 0.5943077802658081),
 ('bagel', 0.5895130038261414)]

In [54]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.80775386095047),
 ('amazed', 0.7797470092773438),
 ('astonished', 0.7748459577560425),
 ('dismayed', 0.7680633068084717),
 ('stunned', 0.7603034973144531),
 ('appalled', 0.7466776371002197)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [55]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('duvet', 0.7086508274078369),
 ('blanket', 0.7016597390174866),
 ('mattress', 0.7002605199813843),
 ('quilt', 0.6868821978569031),
 ('matress', 0.6777950525283813),
 ('pillowcase', 0.6413239240646362),
 ('sheets', 0.6382123827934265),
 ('foam', 0.6322235465049744),
 ('pillows', 0.6320573687553406),
 ('comforter', 0.5972476601600647)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [57]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.76181122646029453

In [12]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [13]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.2682045473522595

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [63]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [77]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
